In [1]:
import marcand as mrc
import nucleo as ncl
print("All marcand librairies and functions imported")

All marcand librairies and functions imported


In [ ]:
# mrc.calculate_all_pos()

# HERE
def gillespie_algorithm_marcand(_total_lenght_, _l_max_, _x_max_, _origin_, _alpha_list_, _p_, _beta_, _n_t_, _epsilon_t_):

    
    # alpha_0 to adjust speed
    _alpha_0_ = 1

    # probabilities
    _p_list_ = _p_

    # beta
    _beta_list_ = np.full(_total_lenght_, _beta_)

    # RESULTS : [MAIN DATA] - matrices of all positions as a function of times
    _results_ = np.empty((_n_t_, int(_x_max_+1)))
    _results_.fill(np.nan)

    # all t, all x
    _all_t_, _all_x_ = [None] * _n_t_, [None] * _n_t_

    #-- Loop for modelling --#
    for _ in range(0,_n_t_) :

        # print('\n #-- new trajectory --#')

        # first term of each line
        _results_[_][0] = 0

        # calibration : begin's by x=0 not x=1
        i0 = 0
        i = 0
        t = 0
        x = folding(_alpha_list_[_], _origin_)

        # lists
        t_list = [t]
        x_list = [x-_origin_]

        #-- Loop for choice of time and reaction --#
        while (x<_x_max_) :

            # Gillespie values
            r_tot = _beta_list_[x] + np.nansum(_p_list_[1:(_l_max_-x)] * _alpha_list_[_][(x+1):_l_max_])

            if r_tot<=0:
                _results_[_][i0:] = t
                break

            t = t - np.log(random())/r_tot + _epsilon_t_                                                           # Next time of reation
            t = np.round(t,4).item()

            # Unhooking or not
            r0 = random()
            if r0<(_beta_list_[x]/r_tot) :
                i = int(np.floor(x))
                _results_[_][i0:int(min(np.floor(_x_max_),i)+1)] = t
                # print("unhook: t =", t,"; i0 =", i0, "i =",i,"; x =",prev_x)
                break

            # Choosing the reaction
            else :
                di = 1                                                                               # ! di begins to 1 : p(0)=0 !
                rp = _beta_list_[x] + _p_list_[di] * _alpha_list_[_][x+di]                                 # Gillespie reaction rate

                while ((rp/r_tot)<r0) and (di<_l_max_-1-x) :                                           # Sum on all possible states
                    di += 1                                                                          # Determining the rank of jump
                    rp += _p_list_[di] * _alpha_list_[_][x+di]                                           # Sum : element per element

            # Updated parameters
            x += di

            # Acquisition of data
            t_list.append(t)
            x_list.append(x-_origin_)

            #------------- Filling -----------#
            if math.isinf(np.floor(x)) :
                i = _x_max_
            else :
                i = np.copy(x)
            # print("t =", t,"; i0 =", i0, "- i =",i,"; x =",x)                                     # Essential line to observe trajectories if needed
            _results_[_][i0:int(min(np.floor(_x_max_),i)+1)] = np.copy(t)
            i0 = i+1
            #---------------------------------#

        # All datas
        _all_t_[_] = t_list
        _all_x_[_] = x_list

    # Update
    _t_max_ = np.floor(max(max(sublist) for sublist in _all_t_))
    
    return _results_, _all_t_, _all_x_, _t_max_   
#--   --#